# 📓 The GenAI Revolution Cookbook

**Title:** 7 Prompt Engineering Techniques to Write Effective AI Prompts

**Description:** Master prompt engineering with actionable frameworks to craft effective, bias-aware prompts, reduce hallucinations, and iterate faster across multimodal tools confidently.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Structured output generation is one of the most common failure modes in production GenAI systems. You send a prompt expecting JSON with specific keys, and the model returns prose, malformed syntax, or extra commentary that breaks your parser. This guide shows you how to design prompts that reliably produce schema-compliant outputs across providers.

## What Problem Are We Solving?

Models trained on diverse text corpora default to fluent prose, not structured data. When you ask for JSON without explicit constraints, the model may wrap the output in markdown fences, add explanatory text, omit required keys, or invent extra fields. This breaks downstream validation, increases token costs, and requires brittle post-processing.

## What's Actually Happening

Language models predict the next token based on learned patterns. Without clear structural cues, they optimize for fluency over format. Three factors drive format drift:

- **Instruction dilution.** Long prompts bury the format requirement in context, reducing its influence on generation.
- **Schema ambiguity.** Vague instructions like "return JSON" don't specify keys, types, or nesting, leaving the model to guess.
- **Output priors.** Training data contains more conversational responses than raw JSON, biasing the model toward prose wrappers.

## How to Fix It

Use explicit delimiters, inline schemas, and role framing to anchor the model's output structure. Here's a minimal pattern that works across providers.

### Before: Vague Format Request

In [ ]:
Extract company names from the text below.

Acme Co. acquired Northwind. ACME has EU ops.

**Output:**

In [ ]:
The companies mentioned are Acme Co. and Northwind.

This fails validation because it's prose, not JSON.

### After: Explicit Schema and Delimiters

In [ ]:
Instructions. Extract companies as JSON key "companies" (array of strings). No extra text.

DATA:
"""
Acme Co. acquired Northwind. ACME has EU ops.
"""

**Output:**

```json
{"companies": ["Acme Co.", "Northwind", "ACME"]}
```

This passes schema validation and is parser-ready.

### Why This Works

- **Delimiters** (triple quotes) separate instructions from user data, preventing the model from treating input text as part of the task description.
- **Inline schema** ("JSON key 'companies' (array of strings)") specifies the exact structure, reducing ambiguity.
- **Explicit constraint** ("No extra text") suppresses conversational wrappers and markdown fences.

### Validation Harness

Test your prompts programmatically to catch drift early. The following harness sends a prompt to OpenAI, validates the output against a JSON schema, and logs results.

In [ ]:
# Install required packages for schema validation and HTTP requests
!pip install requests jsonschema

In [ ]:
import os
import json
import time
import logging
import requests
from jsonschema import validate, ValidationError
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')

keys = ["OPENAI_API_KEY"]
missing = []
for k in keys:
    value = None
    try:
        value = userdata.get(k)
    except SecretNotFoundError:
        pass
    os.environ[k] = value if value is not None else ""
    if not os.environ[k]:
        missing.append(k)
if missing:
    raise EnvironmentError(f"Missing keys: {', '.join(missing)}. Add them in Colab → Settings → Secrets.")
logging.info("All keys loaded.")

SCHEMA = {
    "type": "object",
    "properties": {
        "companies": {
            "type": "array",
            "items": {"type": "string"}
        }
    },
    "required": ["companies"]
}

def call_openai(prompt, model="gpt-4o-mini", temperature=0.0, max_tokens=256):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json"
    }
    body = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=body, headers=headers, timeout=30)
    response.raise_for_status()
    data = response.json()
    try:
        return data["choices"][0]["message"]["content"]
    except (KeyError, IndexError) as e:
        logging.error(f"Unexpected API response structure: {data}")
        raise

def validate_json_output(output, schema):
    obj = json.loads(output)
    validate(instance=obj, schema=schema)
    return obj

def run_tests(test_prompts, schema):
    for idx, prompt in enumerate(test_prompts):
        logging.info(f"Test {idx+1}: Sending prompt to model.")
        try:
            output = call_openai(prompt)
            logging.info(f"Raw model output: {output}")
            obj = validate_json_output(output, schema)
            print("PASS", obj)
        except (json.JSONDecodeError, ValidationError) as e:
            print("FAIL", output[:120] if 'output' in locals() else "No output", e)
        except Exception as e:
            print("ERROR", str(e))
        time.sleep(0.5)

test_prompts = [
    '''Instructions. Extract companies as JSON key companies.
DATA:
"""
Acme Co. acquired Northwind. ACME has EU ops.
"""'''
]

run_tests(test_prompts, SCHEMA)

Run this harness with 10–20 test cases covering edge cases (empty input, special characters, ambiguous entities). Track pass rate and adjust your prompt until you hit 95%+ compliance.

## Key Takeaways

- Use triple-quoted delimiters to separate instructions from user data and prevent instruction leakage.
- Specify the exact JSON structure inline (key names, types, nesting) rather than relying on implicit understanding.
- Add explicit constraints ("No extra text", "Raw JSON only") to suppress conversational wrappers.
- Validate outputs programmatically with a schema and a test suite to catch drift before production.
- Set temperature to 0.0 for deterministic structured outputs and increase max_tokens only if truncation occurs.

This pattern works for extraction, classification, and transformation tasks where schema compliance is non-negotiable. For tasks requiring creativity or multi-step reasoning, combine this approach with chain-of-thought or self-critique patterns covered in separate guides.